## Read data from train.txt and filter it from unwanted patterns


In [2]:
import numpy as np
import torch
import torch.nn as nn
from keras.utils import to_categorical
from Embeddings import Word2Vec, FastText
from Preprocessing import utils, character_encoding
from Models import rnn, lstm, bilstm, rnn_pytorch
import config as conf

config = conf.ConfigLoader().load_config()

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
# VECTOR_SIZE = config['embedding_vector_size ']
VECTOR_SIZE = 10
NUM_TRAIN_LINES = 1

## Preprocessing


Clean data from special characcters


In [3]:
training_set = utils.read_data(f"./Dataset/train.txt")
print("training_set", training_set[0:500])
print('\n')
filtered_training_set = utils.filter_data(training_set)
print("filtered_training_set", filtered_training_set[0:500])


training_set قَوْلُهُ : ( أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ ) قَالَ الزَّرْكَشِيُّ( 14 / 123 )
ابْنُ عَرَفَةَ : قَوْلُهُ : بِلَفْظٍ يَقْتَضِيه كَإِنْكَارِ غَيْرِ حَدِيثٍ بِالْإِسْلَامِ وُجُوبَ مَا عُلِمَ وُجُوبُهُ مِنْ الدِّينِ ضَرُورَةً ( كَإِلْقَاءِ مُصْحَفٍ بِقَذَرٍ وَشَدِّ زُنَّارٍ ) ابْنُ عَرَفَةَ : قَوْلُ ابْنِ شَاسٍ : أَوْ بِفِعْلٍ يَتَضَمَّنُهُ هُوَ كَلُبْسِ الزُّنَّارِ وَإِلْقَاءِ الْمُصْحَفِ فِي صَرِيحِ النَّجَاسَةِ وَالسُّجُودِ لِلصَّنَمِ وَنَحْوِ ذَلِكَ ( وَسِحْرٍ ) مُحَمَّدٌ : قَوْلُ مَالِكٍ و


filtered_training_set قَوْلُهُ أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ قَالَ الزَّرْكَشِيُّ 
ابْنُ عَرَفَةَ قَوْلُهُ بِلَفْظٍ يَقْتَضِيه كَإِنْكَارِ غَيْرِ حَدِيثٍ بِالْإِسْلَامِ وُجُوبَ مَا عُلِمَ وُجُوبُهُ مِنْ الدِّينِ ضَرُورَةً كَإِلْقَاءِ مُصْحَفٍ بِقَذَرٍ وَشَدِّ زُنَّارٍ ابْنُ عَرَفَةَ قَوْلُ ابْنِ شَاسٍ أَوْ بِفِعْلٍ يَتَضَمَّنُهُ هُوَ كَلُبْسِ الزُّنَّارِ وَإِلْقَاءِ الْمُصْحَفِ فِي صَرِيحِ النَّجَاسَةِ وَالسُّجُودِ لِلصَّنَمِ وَنَحْوِ ذَلِكَ وَسِحْرٍ مُحَمَّدٌ قَوْلُ مَالِك

In [4]:
validation_set = utils.read_data(f"./Dataset/val.txt")
print("validation_set", validation_set[0:500])
print('\n')
filtered_validation_set = utils.filter_data(validation_set)
print("filtered_validation_set", filtered_validation_set[0:500])


validation_set ( 27 ) قَوْلُهُ : وَلَا تُكْرَهُ ضِيَافَتُهُ .
( الْفَرْقُ الثَّالِثُ وَالثَّلَاثُونَ بَيْنَ قَاعِدَةِ تَقَدُّمِ الْحُكْمِ عَلَى سَبَبِهِ دُونَ شَرْطِهِ أَوْ شَرْطِهِ دُونَ سَبَبِهِ وَبَيْنَ قَاعِدَةِ تَقَدُّمِهِ عَلَى السَّبَبِ وَالشَّرْطِ جَمِيعًا ) وَتَحْرِيرُهُ أَنَّ الْحُكْمَ إنْ كَانَ لَهُ سَبَبٌ بِغَيْرِ شَرْطٍ فَتَقَدَّمَ عَلَيْهِ لَا يُعْتَبَرُ أَوْ كَانَ لَهُ سَبَبَانِ أَوْ أَسْبَابٌ فَتَقَدَّمَ عَلَى جَمِيعِهَا لَمْ يُعْتَبَرْ أَوْ عَلَى بَعْضِهَا دُونَ بَعْضٍ اُعْتُبِرَ بِنَاءً عَلَى


filtered_validation_set  قَوْلُهُ وَلَا تُكْرَهُ ضِيَافَتُهُ 
 الْفَرْقُ الثَّالِثُ وَالثَّلَاثُونَ بَيْنَ قَاعِدَةِ تَقَدُّمِ الْحُكْمِ عَلَى سَبَبِهِ دُونَ شَرْطِهِ أَوْ شَرْطِهِ دُونَ سَبَبِهِ وَبَيْنَ قَاعِدَةِ تَقَدُّمِهِ عَلَى السَّبَبِ وَالشَّرْطِ جَمِيعًا وَتَحْرِيرُهُ أَنَّ الْحُكْمَ إنْ كَانَ لَهُ سَبَبٌ بِغَيْرِ شَرْطٍ فَتَقَدَّمَ عَلَيْهِ لَا يُعْتَبَرُ أَوْ كَانَ لَهُ سَبَبَانِ أَوْ أَسْبَابٌ فَتَقَدَّمَ عَلَى جَمِيعِهَا لَمْ يُعْتَبَرْ أَوْ عَلَى بَعْضِهَا دُونَ ب

Splite Training data and Validation data into words then separate diacritics from each word


## Feature Extraction


Split training data to sentences and remove diacritics from each sentence


In [ ]:
from torch.utils.data import Dataset, DataLoader
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        # xy = np.loadtxt("./Dataset/train.txt", delimiter=',', dtype=np.float32)
        self.data = torch.from_numpy(data)
        self.labels = torch.from_numpy(labels)
        self.n_samples = data.shape[0]

    def __getitem__(self, index):
        # print("self.data[index]", self.data[index])
        # print("self.labels[index]", self.labels[index])
        return self.data[index], self.labels[index]

    def __len__(self):
        return self.n_samples

In [5]:

sentences = utils.split_data_to_sentences(filtered_training_set)[0:NUM_TRAIN_LINES]
max_sentence_length = utils.get_max_len(sequences=[sentences])
sentences_without_diacritics, sentences_diacritics = character_encoding.RemoveDiacriticFromSentence(sentences)
print(sentences[0])

قَوْلُهُ أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ قَالَ الزَّرْكَشِيُّ 


testing new approach


In [6]:
embedding_model = FastText.FastTextEmbedding(sentences_without_diacritics, vector_size = VECTOR_SIZE)
embedding_model.train()

In [7]:
index = 0
sentence_to_test = sentences_without_diacritics[index]
diacritic_list_to_test = sentences_diacritics[index]

print("sentences_without_diacritics(",len(sentence_to_test),")", sentence_to_test)
print("sentences_diacritics(",len(diacritic_list_to_test),")", character_encoding.map_text_to_diacritic(diacritic_list_to_test))

# words = utils.split_data_to_words(sentence_to_test)
restored_text = character_encoding.restore_diacritics(sentence_to_test, diacritic_list_to_test)
print("original_Text", sentences[index])
print("restored_text", restored_text)

sentences_without_diacritics( 38 ) قوله أو قطع الأول يده إلخ قال الزركشي 
sentences_diacritics( 38 ) ['FATHA', 'SUKUN', 'DAMMA', 'DAMMA', ' ', 'FATHA', 'SUKUN', ' ', 'FATHA', 'FATHA', 'FATHA', ' ', ' ', 'SUKUN', 'FATHA', 'SHADDA_FATHA', 'DAMMA', ' ', 'FATHA', 'FATHA', 'DAMMA', ' ', ' ', 'FATHA', 'SUKUN', ' ', 'FATHA', ' ', 'FATHA', ' ', ' ', ' ', 'SHADDA_FATHA', 'SUKUN', 'FATHA', 'KASRA', 'SHADDA_DAMMA', ' ']
original_Text قَوْلُهُ أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ قَالَ الزَّرْكَشِيُّ 
restored_text قَوْلُهُ أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ قَالَ الزَّرْكَشِيُّ 


In [8]:
def remove_spaces(text, diacritic_list):
    text = list(text)
    copy_diacritic_list = diacritic_list.copy()
    for i,c in enumerate(text):
        if c.isspace():
            text.pop(i)
            copy_diacritic_list.pop(i)
    return text, copy_diacritic_list

In [9]:
import re
def split_data_to_words(data: str) -> list:
    words = re.split(r"[^\S\n]+", data)
    # # remove empty words
    # words = [word for word in words if word]
    return words

In [10]:
# # input is one hot incodeing including spaces and with word embedding
# concatinated_vector_train = []
# for i,sentence in enumerate(sentences_without_diacritics):
#     words = split_data_to_words(sentence)
#     for j, word in enumerate(words):
#         try:
#             word_vec = embedding_model.vector(word)
#         except:
#             print("word not found : \"", word , "\"")
        
#         for k,c in enumerate(word):
#             one_hot = character_encoding.CharToOneHOt(c)
#             v = np.concatenate((word_vec, one_hot), axis=None)
#             concatinated_vector_train.append(v)

#         # add space between words except for the last word
#         if j != len(words) - 1:    
#             one_hot = character_encoding.CharToOneHOt(' ')
#             v = np.concatenate((word_vec, one_hot), axis = None)
#             concatinated_vector_train.append(v) 


# c = 0
# for diacritics in sentences_diacritics:
#     for diacritic in diacritics:
#         c += 1
# # print("vector example :", v)
# print("len concatinated :", len(concatinated_vector_train))
# print("len diacritics   :", c)
# print("Diff = ", len(concatinated_vector_train) - c)
# print("Original sentence :", sentences[0])
# print("First sentence    :", sentences_without_diacritics[0])
# print("First diacritics  :", character_encoding.map_text_to_diacritic(sentences_diacritics[0]))

In [11]:
# input is one hot incodeing including spaces
concatinated_vector_train = []
for i,sentence in enumerate(sentences_without_diacritics):
    sentence_vec = []
    for j,c in enumerate(sentence):
        one_hot = character_encoding.CharToOneHOt(c)
        sentence_vec.append(one_hot)
    concatinated_vector_train.append(sentence_vec)

print("len sentence ", len(sentences_without_diacritics[-1]))
print("len diacritics ", len(sentences_diacritics))
print("sentence ", sentences_without_diacritics[-1])
print("diacritics ", sentences_diacritics[-1])
print("Original sentence :", sentences[0])
print("First sentence    :", sentences_without_diacritics[0])
print("First diacritics  :", character_encoding.map_text_to_diacritic(sentences_diacritics[0]))

len sentence  38
len diacritics  1
sentence  قوله أو قطع الأول يده إلخ قال الزركشي 
diacritics  ['َ', 'ْ', 'ُ', 'ُ', '', 'َ', 'ْ', '', 'َ', 'َ', 'َ', '', '', 'ْ', 'َ', 'َّ', 'ُ', '', 'َ', 'َ', 'ُ', '', '', 'َ', 'ْ', '', 'َ', '', 'َ', '', '', '', 'َّ', 'ْ', 'َ', 'ِ', 'ُّ', '']
Original sentence : قَوْلُهُ أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ قَالَ الزَّرْكَشِيُّ 
First sentence    : قوله أو قطع الأول يده إلخ قال الزركشي 
First diacritics  : ['FATHA', 'SUKUN', 'DAMMA', 'DAMMA', ' ', 'FATHA', 'SUKUN', ' ', 'FATHA', 'FATHA', 'FATHA', ' ', ' ', 'SUKUN', 'FATHA', 'SHADDA_FATHA', 'DAMMA', ' ', 'FATHA', 'FATHA', 'DAMMA', ' ', ' ', 'FATHA', 'SUKUN', ' ', 'FATHA', ' ', 'FATHA', ' ', ' ', ' ', 'SHADDA_FATHA', 'SUKUN', 'FATHA', 'KASRA', 'SHADDA_DAMMA', ' ']


## Building The Model


Preparing Training data to be passed into the `model.train()`


In [12]:
# Convert the training data to the required format
output_size = len(character_encoding.DIACRITICS)
X_train = concatinated_vector_train # Original
# X_train = padded_sequences

y_train = []
for diacritics in sentences_diacritics:
    for diacritic in diacritics:
        index = character_encoding.DIACRITICS.index(diacritic)
        y_train.append(to_categorical(index, num_classes = output_size))

        
y_train = np.array(y_train)
X_train = np.array(X_train)
print("X_train size : ", X_train.shape)
print("y_train size : ", y_train.shape)

training_data = (concatinated_vector_train, sentences_diacritics)
print("sentences size : ", len(training_data[0]))
print("labels size : ", len(training_data[1]))
# assert X_train.shape[0] == y_train.shape[0] # Original
assert X_train.shape[1] == y_train.shape[0]

X_train size :  (1, 38, 38)
y_train size :  (38, 15)
sentences size :  1
labels size :  1


In [ ]:
dataset = CustomDataset(X_train, y_train)
dataloader = DataLoader(dataset=dataset, batch_size=1, shuffle=True, num_workers=2)
dataiter = iter(dataloader)
data = dataiter.next()
features, labels = data
print(features, labels)



Choose Classification model from `config.json file`


In [13]:

# Hyperparameters
input_size = X_train.shape[1]  # Size of your one-hot vector
hidden_size = 128  # Can be any number
output_size = y_train.shape[1]  # Number of classes
num_epochs = 10
learning_rate = 0.001

# Initialize the RNN model, Loss function and Optimizer
model = rnn_pytorch.RNN_Model(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# training_data = list(zip(concatinated_vector_train, sentences_diacritics))

sequence_length = utils.get_max_len(sequences=sentences_without_diacritics)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("trainnig on : ", device)
# Training loop
for epoch in range(num_epochs):
    for i, (sentences, labels) in enumerate(dataloader):  # Assuming you have a PyTorch DataLoader
        print("sentences : ", sentences) # ex : [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,1], [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]]
        print("labels : ", labels) # ex : ['`', '``]
        sentences = sentences.reshape(-1, sequence_length, input_size).to(device)
        labels = [character_encoding.DIACRITICS.index(char) for char in labels]
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(sentences)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

c:\Users\marky\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


trainnig on :  cpu
sentence :  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

RuntimeError: For unbatched 2-D input, hx should also be 2-D but got 3-D tensor

## Model Evaluation


Preparing Validation data to be passed into the `model.evaluate()`


# Testing


testing on a given sentence


In [ ]:
# # Predict the diacritics of the validation data
# target_text = sentences[0]
# _, original_diacritics = character_encoding.remove_diacritics(target_text, is_sentence=True)
# sentence_test = sentences_without_diacritics[0]

# sentences = utils.split_data_to_sentences(sentence_test)
# # get one hot encoding for each character in sentence_to_test
# x_test = []
# batch_concatinated_vector_train = []
# for i,sentence in enumerate([sentence_test]):
#     sentence_vec = []
#     words = split_data_to_words(sentence)
#     for j, word in enumerate(words):
#         try:
#             word_vec = embedding_model.vector(word)
#         except:
#             print("word not found : \"", word , "\"")
        
#         for k,c in enumerate(word):
#             one_hot = character_encoding.CharToOneHOt(c)
#             v = np.concatenate((word_vec, one_hot), axis=None)
#             sentence_vec.append(v)

#         # add space between words except for the last word
#         if j != len(words) - 1:    
#             one_hot = character_encoding.CharToOneHOt(' ')
#             v = np.concatenate((word_vec, one_hot), axis = None)
#             sentence_vec.append(v)
#     batch_concatinated_vector_train.append(sentence_vec)

# x_test = utils.padding(batch_concatinated_vector_train)
# # print("x_test : ", x_test)
# x_test = np.array(x_test)
# print("x_test len : ", len(sentence_test))
# print("X_test size : ", x_test.shape)

In [ ]:
# Predict the diacritics of the validation data
target_text = sentences[0]
_, original_diacritics = character_encoding.remove_diacritics(target_text, is_sentence=True)
sentence_test = sentences_without_diacritics[0]

sentences = utils.split_data_to_sentences(sentence_test)
# get one hot encoding for each character in sentence_to_test
x_test = []
for i,sentence in enumerate([sentence_test]):
    words = split_data_to_words(sentence)
    for j, word in enumerate(words):
        try:
            word_vec = embedding_model.vector(word)
        except:
            print("word not found : \"", word , "\"")
        
        for k,c in enumerate(word):
            one_hot = character_encoding.CharToOneHOt(c)
            v = np.concatenate((word_vec, one_hot), axis=None)
            x_test.append(v)

        # add space between words except for the last word
        if j != len(words) - 1:    
            one_hot = character_encoding.CharToOneHOt(' ')
            v = np.concatenate((word_vec, one_hot), axis = None)
            x_test.append(v) 

# print("x_test : ", x_test)
x_test = np.array(x_test)
print("x_test len : ", len(sentence_test))
print("X_test size : ", x_test.shape)

x_test len :  38
X_test size :  (38, 48)


In [ ]:
y_pred = model.predict(x_test)

# # Make predictions at each timestamp
# y_pred = []

# # Iterate over each timestamp
# for i in range(len(sentence_test)):
#     # Get the prediction at the current timestamp
#     predictions = model.predict(x_test[:, :i+1])
#     y_pred.append(predictions)

2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step


In [ ]:
# something is wrong here
print("X : ", sentence_test)
print("Len X  : ", len(sentence_test))
print("Len Y predicted  : ", len(y_pred))


predicted_diacritics = []
for i in range(len(y_pred)):
    predicted_diacritics.append(character_encoding.DIACRITICS[np.argmax(y_pred[i])])

# Print the predicted diacritics
restored_text = character_encoding.restore_diacritics(sentence_test, predicted_diacritics)
print("Original Text : ", target_text)
print("Restored Text : ", restored_text)

X :  قوله أو قطع الأول يده إلخ قال الزركشي 
Len X  :  38
Len Y predicted  :  38
Original Text :  قَوْلُهُ أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ قَالَ الزَّرْكَشِيُّ 
Restored Text :  قَوْلُهُ أَوْ قَطَعَ الُأَوَّلُ يَدَهُ إلَخْ قَالَ الزَّرْكَشِيُّ 


In [ ]:
print("Original diacritics  : ", character_encoding.map_text_to_diacritic(original_diacritics))
print("Predicted diacritics : ", character_encoding.map_text_to_diacritic(predicted_diacritics))

Original diacritics  :  ['FATHA', 'SUKUN', 'DAMMA', 'DAMMA', ' ', 'FATHA', 'SUKUN', ' ', 'FATHA', 'FATHA', 'FATHA', ' ', ' ', 'SUKUN', 'FATHA', 'SHADDA_FATHA', 'DAMMA', ' ', 'FATHA', 'FATHA', 'DAMMA', ' ', ' ', 'FATHA', 'SUKUN', ' ', 'FATHA', ' ', 'FATHA', ' ', ' ', ' ', 'SHADDA_FATHA', 'SUKUN', 'FATHA', 'KASRA', 'SHADDA_DAMMA', ' ']
Predicted diacritics :  ['FATHA', 'SUKUN', 'DAMMA', 'DAMMA', ' ', 'FATHA', 'SUKUN', ' ', 'FATHA', 'FATHA', 'FATHA', ' ', ' ', 'DAMMA', 'FATHA', 'SHADDA_FATHA', 'DAMMA', ' ', 'FATHA', 'FATHA', 'DAMMA', ' ', ' ', 'FATHA', 'SUKUN', ' ', 'FATHA', ' ', 'FATHA', ' ', ' ', ' ', 'SHADDA_FATHA', 'SUKUN', 'FATHA', 'KASRA', 'SHADDA_DAMMA', ' ']


In [ ]:
def diacritics_error_rate(original_diacritics, predicted_diacritics):
    error = 0
    for i in range(len(original_diacritics)):
        if original_diacritics[i] != predicted_diacritics[i]:
            error += 1
    return error / len(original_diacritics) * 100, error

In [ ]:
diacritic_error_rate, number_of_mis_classified = diacritics_error_rate(original_diacritics, predicted_diacritics)
print("Diacritic Error Rate = ", diacritic_error_rate, "%")
print("Diacritic Correct Rate = ", 100 - diacritic_error_rate, "%")
print("Number of Misclassified = ", number_of_mis_classified, "out of", len(original_diacritics))

Diacritic Error Rate =  2.631578947368421 %
Diacritic Correct Rate =  97.36842105263158 %
Number of Misclassified =  1 out of 38
